In [1]:
import numpy as np
from matplotlib import pyplot as plt
from PIL import Image
from matplotlib import image
import os

In [2]:
#loading 240 images for training & 160 for testing

train_image_list=[]
test_image_list=[]
for i in range(1,41):
    j=1
    s = os.listdir('s'+str(i)+'/')
    for image in s:
        if(j<=6):
            im = Image.open('s'+str(i)+'/' + image)
            train_image_list.append(np.asarray(im).flatten())
        else:
            im = Image.open('s'+str(i)+'/' + image)
            test_image_list.append(np.asarray(im).flatten())
        j+=1

In [3]:
print(len(train_image_list))
print(len(test_image_list))

240
160


In [4]:
# 1. Generating the face dataset for training (60%)

Face_Db = np.array(train_image_list).T
print(Face_Db.shape)

# 2. Mean Calculation

mn,p = Face_Db.shape
M = np.zeros((mn,1))
for i in range(mn):
        for j in range(p):
            M[i] += Face_Db[i][j]
        M[i] /= p
print(M.shape)

# 3. Do mean Zero

delt = np.zeros((mn,p))
for i in range(mn):
    for j in range(p):
        delt[i][j]=Face_Db[i][j]-M[i]

    
# 4. Covariance of delt (mean aligned faces)

def cov(delt):
    row,col=delt.shape
    covm = np.zeros((col,col))        
    for z in range(col):
        for y in range(col):
            covm[z][y] = (1/row) * np.dot(delt[:,y].T,delt[:,z])                   
    return covm

cov_matrix = cov(delt)

# 5. eigenvalue and eigenvector decomposition (sorted)

eigenValues, eigenVectors = np.linalg.eig(cov_matrix)
idx = eigenValues.argsort()[::-1]   
eigen_val = eigenValues[idx]
eigen_vector = eigenVectors[:,idx]
eigen_val = np.diag(eigen_val)
print("...Eigen Values...")
print(eigen_val.shape)
print()
print("...Eigen Vector...")
print(eigen_vector.shape)
print()

# 6. Finding the best direction (Generation of feature vectors)

k=100
Feature_vector = eigen_vector[:,0:k]
print(Feature_vector.shape)

# 7. Generating Eigenfaces

Eigen_faces = np.dot(Feature_vector.T,delt.T)
print(Eigen_faces.shape)

# 8. Generating Signature of Each Face

Signature = np.dot(Eigen_faces,delt)
print(Signature.shape)



(10304, 240)
(10304, 1)
...Eigen Values...
(240, 240)

...Eigen Vector...
(240, 240)

(240, 100)
(100, 10304)
(100, 240)


In [5]:
P=len(train_image_list)
n=6 #no. of images per person
number_of_classes=P//n
print(number_of_classes)

40


In [12]:
def meann(Face):
    mn,p = Face.shape
    M = np.zeros((mn,1))
    for i in range(mn):
        for j in range(p):
            M[i] += Face[i][j]
        M[i] /= p
    return M

def sub(X,M):
    res = X
    n,c = X.shape
    
    for i in range(n):
        for j in range(c):
            res[i][j] = res[i][j] - M[i]
    
    return res

In [13]:
# 3.

mean_each_class = np.zeros((k,number_of_classes))

for i in range(number_of_classes):
    pf=Signature[:,6*i:6*i+6]
    mean_each_class[:,i] = meann(pf).T
    
mean_pf = meann(pf)
mean_each_class.shape

(100, 40)

In [14]:
mean_pf.shape

(100, 1)

In [22]:
# 4. within class scatter matrix (SW)

SW = np.zeros((k,k))
for i in range(number_of_classes):
    V_mu = sub(pf,mean_each_class[:,i])
    SW += (np.dot(V_mu,V_mu.T))
    
SW.shape


(100, 100)

In [ ]:
# Between Class Scatter Matrix
SB = np.zeros((k,k))
for i in range(number_of_classes):
    M_i = np.reshape(mean_each_class[:,i],(1,mean_each_class[:,i].size)).T
    SB += (np.dot(sub(M_i,mean_PF),M_i.T))

SB.shape